In [39]:
import io
import xml.etree.ElementTree as ET
import pandas as pd
import helper
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import numpy as np
import os
from gensim import corpora, models, similarities 
from gensim.corpora.mmcorpus import MmCorpus

time: 2.97 ms


In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 692 ms


In [41]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 783 ms


In [42]:
#%unload_ext autotime

time: 393 ms


[autoreload of autotime failed: Traceback (most recent call last):
  File "/home/jshoun01/.virtualenvs/py27/local/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named autotime
]


In [43]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

time: 456 ms


In [44]:
ls data

data@
time: 722 ms


In [45]:
data_dir='data'

time: 222 ms


## Normalizer

In [46]:
ag_preprocesser = helper.Normalizer(stem=False\
                                 ,stopword_removal=True\
                                 ,strip_html_tags=True
                                   ,filter_parts_of_speech=True
                                    ,min_word_length=1
                                   )

time: 426 ms


## Loading Streaming

In [47]:
post_file=os.path.join(data_dir,'Posts.xml')
posts_corpus = helper.XmlCorpus(post_file
                                ,clip_docs=100000
                                #,clip_docs=1000
                                ,body_field='Body'
                                ,normalizer=ag_preprocesser)
posts_corpus.finalize_dictionary()


time: 4min 6s


In [ ]:
posts_corpus.

In [29]:
sample_msg='<p>A Vista virtual machine I use only has a 10 GB virtual hard disk -- and I\'m worried about it running out of space over time.</p>\n\n<p>I used <a href="http://www.jam-software.com/treesize_free/">TreeSize</a> to check for the culprits..</p>\n\n<p><img src="http://i.stack.imgur.com/uGfhQ.png" alt="enter image description here"></p>\n\n<p>And the primary culprit is that <strong><code>/winsxs</code> or "Windows Side-By-Side" folder</strong>.</p>\n\n<p><img src="http://i.stack.imgur.com/DWYOo.png" alt="enter image description here"></p>\n\n<p>What is the Windows Side By Side folder? Well, that\'s <a href="http://blogs.technet.com/askcore/archive/2008/09/17/what-is-the-winsxs-directory-in-windows-2008-and-windows-vista-and-why-is-it-so-large.aspx">explained pretty well here</a>.</p>\n\n<blockquote>\n  <p>All of the components in the operating system are found in the WinSxS folder \u2013 in fact we call this location the component store.  Each component has a unique name that includes the version, language, and processor architecture that it was built for.  The WinSxS folder is the only location that the component is found on the system, all other instances of the files that you see on the system are \u201cprojected\u201d by hard linking from the component store.  Let me repeat that last point \u2013 there is only one instance (or full data copy) of each version of each file in the OS, and that instance is located in the WinSxS folder.   So looked at from that perspective, the WinSxS folder is really the entirety of the whole OS, referred to as a "flat" in down-level operating systems.  This also accounts for why you will no longer be prompted for media when running operations such as System File Checker (SFC), or when installing additional features and roles.</p>\n  \n  <p>That explains why the folder starts off big, but not why it gets larger over time \u2013 the answer to that question is servicing.   In previous versions of Windows the atomic unit of servicing was the file, in Windows Vista it\u2019s the component.  When we update a particular binary we release a new version of the whole component, and that new version is stored alongside the original one in the component store.  The higher version of the component is projected onto the system, but the older version in the store isn\u2019t touched.  The reason for that is the third part of why the component store gets so large.</p>\n</blockquote>\n\n<p>Lots of other people <a href="http://blog.tiensivu.com/aaron/archives/1306-Demystifying-the-WinSxS-directory-in-Windows-XP,-Vista-and-Server-20032008.html">complaining about the size of the /winsxs folder, too</a>:</p>\n\n<blockquote>\n  <p>A coworker recently received his every two year work laptop refresh and was in the middle of loading his new laptop with Vista. For some reason, his hard drive C: was partitioned to only 25GB and the rest of the space reserved for a much larger drive D:.</p>\n  \n  <p>Anyway, after loading many Windows Updates, he was left with hardly any drive space left on C:, and for some reason, the WinSxS directory had ballooned to over 8GB.</p>\n  \n  <p>While I knew what the WinSxS (Windows Side-by-Side) directory was for, I wasn\'t quite sure why that directory would explode in size so fast with normal operating system updates.</p>\n</blockquote>\n\n<p>I don\'t really care about having all these duplicate files for safety, but I <em>do</em> care about the VM exploding due to an out of disk space error. So.. <strong>can /winsxs be safely pared down in any way?</strong> <a href="http://blogs.msdn.com/jonwis/archive/2007/01/02/deleting-from-the-winsxs-directory.aspx">It doesn\'t seem that it can be</a>.. </p>\n\n<blockquote>\n  <p><strong>Administrators should not, for any reason, take it upon themselves to clean out the /winsxs directory</strong> - doing so may prevent Windows Update and MSI from functioning properly afterwards.  Preventing accidental deletion from the directory is accomplished by putting a strong security descriptor on the directory that inherits to its children.</p>\n</blockquote>\n\n<p>If this /winsxs folder gets any larger, I\'m going to have to re-image this virtual machine.. which I <em>really</em> don\'t want to do!</p>\n'

ag_preprocesser.process(sample_msg)[0:5]

[u'Vista', u'virtual', u'machine', u'use', u'GB']

time: 17.9 ms


In [30]:
"""
num_documents_processed=0
counter=0
for vector in posts_corpus: # load one vector into memory at a time
    print ("Item %s"%(counter))
    print(vector)
    print()
    counter=counter+1
"""

'\nnum_documents_processed=0\ncounter=0\nfor vector in posts_corpus: # load one vector into memory at a time\n    print ("Item %s"%(counter))\n    print(vector)\n    print()\n    counter=counter+1\n'

time: 661 ms


In [31]:
print ("Number of docs in collection: %s"%(len(posts_corpus)))

Number of docs in collection: 100001
time: 911 ms


In [32]:
#gensim.models.LdaMulticore
"""
lda = gensim.models.LdaModel(corpus, num_topics=5, 
                            id2word=dictionary, 
                            update_every=5, 
                            chunksize=10000,
                            passes=100)
"""

num_topics = 20

#lda = gensim.models.LdaModel(posts_corpus, num_topics=5, 
#                            id2word=posts_corpus.dictionary
#                                ,passes=2
#                                )

lda = gensim.models.LdaMulticore(posts_corpus, num_topics=num_topics, 
                            id2word=posts_corpus.dictionary
                                 ,passes=3
                                )

100000
100000
100000
time: 13min 16s


In [33]:
lda.show_topics(num_topics)

[u'0.024*drive + 0.023*files + 0.023*file + 0.013*partition + 0.013*disk + 0.011*Windows + 0.010*boot + 0.008*hard + 0.008*system + 0.008*use',
 u'0.038*gt + 0.028*lt + 0.023*file + 0.010*directory + 0.008*use + 0.008*Firefox + 0.006*user + 0.006*like + 0.005*root + 0.005*set',
 u'0.029*command + 0.016*line + 0.016*use + 0.014*amp + 0.013*key + 0.012*script + 0.008*file + 0.008*like + 0.007*http + 0.007*set',
 u'0.015*Google + 0.012*search + 0.011*use + 0.009*http + 0.008*like + 0.007*would + 0.007*using + 0.007*one + 0.006*get + 0.005*work',
 u'0.013*machine + 0.012*Windows + 0.010*virtual + 0.009*laptop + 0.009*use + 0.009*PC + 0.008*would + 0.007*time + 0.007*running + 0.006*XP',
 u'0.016*user + 0.015*password + 0.013*computer + 0.011*router + 0.011*wireless + 0.010*access + 0.010*account + 0.010*network + 0.008*use + 0.008*server',
 u'0.050*Windows + 0.029*install + 0.017*version + 0.016*Ubuntu + 0.015*XP + 0.012*installed + 0.011*Linux + 0.009*Vista + 0.008*use + 0.008*package',
 

time: 26.7 ms


In [34]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print([str(word) for word in i])
    print()

['card', 'II', 'video', 'one', 'cable', 'cards', 'sound', 'fglrx', 'USB', 'output', 'use', 'would', 'two', 'device', 'port', 'DVI', 'graphics', 'like', 'computer', 'VGA']
()
['user', 'password', 'computer', 'router', 'wireless', 'access', 'account', 'network', 'use', 'server', 'like', 'one', 'using', 'connect', 'get', 'would', 'want', 'log', 'remote', 'set']
()
['screen', 'monitor', 'image', 'resolution', 'display', 'like', 'using', 'use', 'color', 'monitors', 'one', 'size', 'would', 'want', 'need', 'also', 'background', 'nbsp', 'desktop', 'make']
()
['power', 'problem', 'USB', 'one', 'see', 'computer', 'kernel', 'device', 'try', 'case', 'could', 'get', 'may', 'fan', 'issue', 'would', 'supply', 'motherboard', 'like', 'drive']
()
['window', 'Windows', 'use', 'Option', 'windows', 'screen', 'open', 'application', 'run', 'Device', 'mouse', 'work', 'Manager', 'Screen', 'bit', 'using', 'Task', 'version', 'one', 'applications']
()
['Google', 'search', 'use', 'http', 'like', 'would', 'using', 

In [35]:
lda.show_topics()

[u'0.015*Google + 0.012*search + 0.011*use + 0.009*http + 0.008*like + 0.007*would + 0.007*using + 0.007*one + 0.006*get + 0.005*work',
 u'0.017*files + 0.014*use + 0.010*file + 0.009*like + 0.009*free + 0.009*video + 0.008*would + 0.008*iTunes + 0.007*want + 0.006*software',
 u'0.027*card + 0.016*II + 0.013*video + 0.012*one + 0.012*cable + 0.010*cards + 0.009*sound + 0.008*fglrx + 0.008*USB + 0.008*output',
 u'0.031*Windows + 0.013*network + 0.009*XP + 0.008*computer + 0.007*Internet + 0.007*Explorer + 0.007*use + 0.006*machine + 0.006*connection + 0.005*running',
 u'0.017*driver + 0.016*drivers + 0.009*Windows + 0.008*Intel + 0.007*hardware + 0.007*card + 0.007*get + 0.006*BIOS + 0.006*system + 0.006*laptop',
 u'0.038*gt + 0.028*lt + 0.023*file + 0.010*directory + 0.008*use + 0.008*Firefox + 0.006*user + 0.006*like + 0.005*root + 0.005*set',
 u'0.021*power + 0.019*problem + 0.016*USB + 0.008*one + 0.008*see + 0.007*computer + 0.007*kernel + 0.006*device + 0.006*try + 0.006*case',
 u

time: 618 ms


In [36]:
model_serialized_path='lda_topic_%s.lda'%(num_topics)
lda.save(model_serialized_path) 

time: 777 ms


In [37]:
lda = gensim.models.LdaMulticore.load(model_serialized_path)

time: 594 ms


In [38]:
lda.show_topics()

[u'0.050*Windows + 0.029*install + 0.017*version + 0.016*Ubuntu + 0.015*XP + 0.012*installed + 0.011*Linux + 0.009*Vista + 0.008*use + 0.008*package',
 u'0.016*user + 0.015*password + 0.013*computer + 0.011*router + 0.011*wireless + 0.010*access + 0.010*account + 0.010*network + 0.008*use + 0.008*server',
 u'0.013*machine + 0.012*Windows + 0.010*virtual + 0.009*laptop + 0.009*use + 0.009*PC + 0.008*would + 0.007*time + 0.007*running + 0.006*XP',
 u'0.017*files + 0.014*use + 0.010*file + 0.009*like + 0.009*free + 0.009*video + 0.008*would + 0.008*iTunes + 0.007*want + 0.006*software',
 u'0.011*Media + 0.010*page + 0.008*device + 0.007*print + 0.007*git + 0.006*Center + 0.006*Windows + 0.006*Version + 0.005*amp + 0.005*Address',
 u'0.021*click + 0.012*select + 0.010*menu + 0.009*Click + 0.009*button + 0.008*want + 0.008*file + 0.008*tab + 0.007*text + 0.007*right',
 u'0.038*gt + 0.028*lt + 0.023*file + 0.010*directory + 0.008*use + 0.008*Firefox + 0.006*user + 0.006*like + 0.005*root + 0

time: 900 ms
